In [13]:
conda install -c conda-forge dask-image

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.12
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /gext/nezar.lheimeur/hduser/conda/envs/spark-env

  added / updated specs:
    - dask-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    dask-image-0.5.0           |     pyhd8ed1ab_1          41 KB  conda-forge
    openssl-1.1.1j             |       h7f98852_0         2.1 MB  conda-forge
    pims-0.5                   |     pyh9f0ad1d_1          66 KB  conda-forge
    slicerator-1.0.0           |             py_0          12 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  dask-image       

In [1]:
# Importing required libraries
import imageio
import numpy as np
from dask_image import imread
from dask import array as darr
#import matplotlib.pyplot as plt
from scipy.ndimage.filters import median_filter


In [2]:
def partial_median(block):
    return median_filter(block, footprint=np.ones((3,3,1)))


In [4]:
# Defined Constants
img = "lena_noisy.jpg"

x_block_size = 8
y_block_size = 8
#noisy= imageio.imread(img)
noisy = imread.imread(img)

chunked = noisy[0].rechunk(chunks=(x_block_size, y_block_size, 3))

#Ghost-Cells Creation
extended = darr.overlap.overlap(chunked, depth={0:1,1:1}, boundary='nearest')

#Apply non-linear kernel over parallel blocks
filtered = extended.map_blocks(partial_median)

result = darr.overlap.trim_internal(filtered, {0:1,1:1})

# Write out resulting image
imageio.imwrite("lena_filter_dask.jpg", result)